# Imports

In [102]:
import pandas as pd
import numpy as np
import json

# Dataset Analyses

In [90]:
# df_init = pd.read_csv("../data/2wiki_1k.csv")
df_init = pd.read_parquet('../data/train_2wiki.parquet')
df_init.drop(columns = ['_id', 'type', 'evidences'], axis = 1, inplace=True)
df_init= df_init.loc[:100].copy()
df_init.head()
# eval(df.context[2])

,question,context,supporting_facts,answer
0,Are director of film Move (1970 Film) and dire...,"[[""Stuart Rosenberg"", [""Stuart Rosenberg (Augu...","[[""Move (1970 film)"", 0], [""M\u00e9diterran\u0...",no
1,Do both films The Falcon (Film) and Valentin T...,"[[""The Falcon Takes Over"", [""The Falcon Takes ...","[[""The Falcon (film)"", 0], [""Valentin the Good...",no
2,"Which film whose director is younger, Charge I...","[[""Danger: Diabolik"", [""Danger:"", ""Diabolik i...","[[""Charge It to Me"", 1], [""Danger: Diabolik"", ...",Danger: Diabolik
3,What is the date of birth of Mina Gerhardsen's...,"[[""Pamela Jain"", [""Pamela Jain is an Indian pl...","[[""Mina Gerhardsen"", 1], [""Rune Gerhardsen"", 0]]",13 June 1946
4,What nationality is the director of film Weddi...,"[[""Weekend in Paradise (1931 film)"", [""Weekend...","[[""Wedding Night in Paradise (1950 film)"", 0],...",Hungarian


In [91]:
def get_supp_sentences(context, supporting_facts):
    supp_sentences = []
    support = dict(supporting_facts)
    for doc in context: 
        if doc[0] in support.keys():
            supp_sentences.append(doc[1][support[doc[0]]]) 
    return supp_sentences

df_init['supporting_sentences'] = df_init[['context', 'supporting_facts']].apply(lambda x: get_supp_sentences(eval(x['context']), eval(x['supporting_facts'])), axis = 1)

In [97]:
df_init

,question,context,supporting_facts,answer,supporting_sentences
0,Are director of film Move (1970 Film) and dire...,"[[""Stuart Rosenberg"", [""Stuart Rosenberg (Augu...","[[""Move (1970 film)"", 0], [""M\u00e9diterran\u0...",no,"[Stuart Rosenberg (August 11, 1927 – March 15,..."
1,Do both films The Falcon (Film) and Valentin T...,"[[""The Falcon Takes Over"", [""The Falcon Takes ...","[[""The Falcon (film)"", 0], [""Valentin the Good...",no,[In 1942 he joined Young Communist League of Y...
2,"Which film whose director is younger, Charge I...","[[""Danger: Diabolik"", [""Danger:"", ""Diabolik i...","[[""Charge It to Me"", 1], [""Danger: Diabolik"", ...",Danger: Diabolik,[Diabolik is a 1968 action film directed and ...
3,What is the date of birth of Mina Gerhardsen's...,"[[""Pamela Jain"", [""Pamela Jain is an Indian pl...","[[""Mina Gerhardsen"", 1], [""Rune Gerhardsen"", 0]]",13 June 1946,[She is the daughter of Rune Gerhardsen and To...
4,What nationality is the director of film Weddi...,"[[""Weekend in Paradise (1931 film)"", [""Weekend...","[[""Wedding Night in Paradise (1950 film)"", 0],...",Hungarian,[Wedding Night in Paradise is a 1950 West Ger...
...,...,...,...,...,...
96,Do Mireille Delunsch and Richard J. Zaborski s...,"[[""Richard J. Zaborski"", [""Richard J. Zaborski...","[[""Mireille Delunsch"", 1], [""Richard J. Zabors...",no,[Richard J. Zaborski( 1927 – 2001) was an Amer...
97,Where was the composer of film Marikkunnilla N...,"[[""Henri Verdun"", [""Henri Verdun( 1895\u201319...","[[""Marikkunnilla Njan"", 2], [""Raveendran"", 0]]",Kulathupuzha,"[Madhavan Raveendran , (born Kulathupuzha, Kol..."
98,Where was the director of film The Sporting Du...,"[[""Ian Barry (director)"", [""Ian Barry is an Au...","[[""The Sporting Duchess (1915 film)"", 0], [""Ba...",New York,[The Sporting Duchess is a lost 1915 silent fi...
99,Do both Something Always Happens and The Great...,"[[""Michael Powell"", [""Michael Latham Powell( 3...","[[""The Great Impersonation (1921 film)"", 0], [...",no,"[Through their production company"" The Archers..."


# Results Analysis

In [201]:
df_res = pd.read_csv('../Experiment_data/2wiki/results_SENTENCE.csv')
df_res.head()

,query,context,provided_answer,lds,lds_compar,rmse,r2,topk_probability,topk_divergence,method_scores
0,Are director of film Move (1970 Film) and dire...,"[[['Stuart Rosenberg (August 11, 1927 – March ...","No, they are not from the same country.","[[[{'ContextCite364': 0.7072302558398219}, {'F...","[[[{'FM_WeightsD364': 0.5323692992213571}, {'F...","[[[{'ContextCite364': 0.46114171482852634}, {'...","[[[{'ContextCite364': 0.43089521879098347}, {'...","[[{'ContextCite364': {1: 0.6225537061691284, 2...","[[{'ContextCite364': {1: 0.1275172300847771, 2...","[{'ContextCite364': array([ 0.22018592, 0.187..."
1,Do both films The Falcon (Film) and Valentin T...,[[['Vatroslav Mimica( born 25 June 1923) is a ...,"No, they have different directors from differe...","[[[{'ContextCite364': 0.8905450500556173}, {'F...","[[[{'FM_WeightsD364': 0.846496106785317}, {'FM...","[[[{'ContextCite364': 1.0840967851312302}, {'F...","[[[{'ContextCite364': 0.6656356967883501}, {'F...","[{'ContextCite364': {1: 2.5072808265686035, 2:...","[{'ContextCite364': {1: 0.2611975879408419, 2:...","[{'ContextCite364': array([-0.0613658 , -0. ..."
2,"Which film whose director is younger, Charge I...","[[['Danger:', 'Diabolik is a 1968 action film...",Charge It To Me.,"[[[{'ContextCite364': 0.793103448275862}, {'FB...","[[[{'FM_WeightsD364': 0.7152391546162402}, {'F...","[[[{'ContextCite364': 2.6373913858414806}, {'F...","[[[{'ContextCite364': 0.5513557373072626}, {'F...","[{'ContextCite364': {1: 1.6020777225494385, 2:...","[{'ContextCite364': {1: 0.24474250542698428, 2...","[{'ContextCite364': array([ 0.26004221, -0.461..."
3,What is the date of birth of Mina Gerhardsen's...,[[['Mina Gerhardsen (born 14 September 1975) i...,13 June 1946.,"[[[{'ContextCite364': 0.8696329254727476}, {'F...","[[[{'FM_WeightsD364': 0.8731924360400445}, {'F...","[[[{'ContextCite364': 6.243895980942368}, {'FB...","[[[{'ContextCite364': 0.888795668684583}, {'FB...","[{'ContextCite364': {1: 22.160571217536926, 2:...","[{'ContextCite364': {1: 2.550866632722318, 2: ...","[{'ContextCite364': array([-6.21530700e-01, 3..."
4,What nationality is the director of film Weddi...,[[['Wedding Night in Paradise is a 1950 West ...,Hungarian.,"[[[{'ContextCite364': 0.907007786429366}, {'FB...","[[[{'FM_WeightsD364': 0.9056729699666296}, {'F...","[[[{'ContextCite364': 2.698198370638433}, {'FB...","[[[{'ContextCite364': 0.906711234314656}, {'FB...","[{'ContextCite364': {1: 3.0173943787813187, 2:...","[{'ContextCite364': {1: 0.6523137472686358, 2:...","[{'ContextCite364': array([ 8.31808734e+00, 6..."


## Flag Correct Responses

In [202]:
df_res.rename({'query': 'question'}, axis = 1, inplace = True)

In [203]:
df_res = pd.merge(df_res, df_init[['answer', 'supporting_sentences', 'question']], on = 'question')

In [204]:
system_prompt = system_prompt = system_prompt = """You are a helpful assistant that evaluates whether two answers express the same meaning.

You will be provided with:
- A ground truth answer
- A predicted answer

Your task is to compare them and determine if the **predicted answer conveys the same meaning** as the **ground truth answer**, even if it uses different words or more elaboration. Minor differences in phrasing, length, or detail are acceptable as long as the core meaning is preserved.

Your output must be one of the following:
- `True` — if the predicted answer has the same meaning as the ground truth answer.
- `False` — if the predicted answer significantly differs in meaning or introduces incorrect information.

Return **only** `True` or `False`. Do not include any explanations or extra text.

Example 1:
Ground Truth Answer: "Water boils at 100 degrees Celsius."
Predicted Answer: "At 100°C, water reaches its boiling point."
Expected Output: True

Example 2:
Ground Truth Answer: "The capital of Japan is Tokyo."
Predicted Answer: "Tokyo is the capital city of Japan."
Expected Output: True

Example 3:
Ground Truth Answer: "Photosynthesis is how plants make food using sunlight."
Predicted Answer: "Photosynthesis helps animals digest food using sunlight."
Expected Output: False

Example 4:
Ground Truth Answer: "She won the race."
Predicted Answer: "She participated in the race."
Expected Output: False

Do not provide explanations—only output `True` or `False`."""

In [205]:
from google.oauth2 import service_account
import vertexai
from vertexai.generative_models import GenerativeModel
 
vertexai.init(
    project="oag-ai",
    credentials=service_account.Credentials.from_service_account_file("google-credentials.json"),
)
 
evaluate_answer = GenerativeModel(
    model_name="gemini-2.0-flash",
    system_instruction= system_prompt,
)#model = "publishers/google/models/gemini-2.0-flash-thinking-exp-01-21"
 
def prompt_just_text(prompt: str,temperature=0.0) -> str:
    return evaluate_answer.generate_content(
        generation_config={
            "temperature": temperature,
            "response_mime_type": "application/json",
        },
        contents=[
            prompt
        ],
    ).text
 

In [206]:
results = []
def evaluate(provided: str, ground_truth:str):
    template_prompt = f"""Evaluate the provided answer using the ground truth answer, is the provided answer correct?:
    Provided answer: {provided}
    Ground Truth: {ground_truth}"""
   
    response = prompt_just_text(template_prompt)
    print("LLM Response: ",  response)
    return response
 
for i in df_res.index:
    res = evaluate(df_res.loc[i].provided_answer, df_res.loc[i].answer)
    results.append(res)

LLM Response:  True
LLM Response:  True
LLM Response:  False
LLM Response:  True
LLM Response:  True
LLM Response:  False
LLM Response:  True
LLM Response:  True
LLM Response:  True
LLM Response:  True
LLM Response:  True
LLM Response:  False
LLM Response:  True
LLM Response:  False
LLM Response:  True
LLM Response:  True
LLM Response:  True
LLM Response:  False
LLM Response:  True
LLM Response:  False
LLM Response:  True
LLM Response:  True
LLM Response:  True
LLM Response:  False
LLM Response:  True
LLM Response:  True
LLM Response:  True
LLM Response:  False
LLM Response:  True
LLM Response:  True
LLM Response:  True
LLM Response:  True
LLM Response:  True
LLM Response:  True
LLM Response:  True
LLM Response:  True
LLM Response:  False
LLM Response:  True
LLM Response:  True
LLM Response:  False
LLM Response:  True
LLM Response:  True
LLM Response:  True
LLM Response:  True
LLM Response:  True
LLM Response:  True
LLM Response:  False
LLM Response:  True
LLM Response:  True
LLM Respo

In [207]:
df_res['flag_correct'] = results
df_res['flag_correct'] = df_res.flag_correct.apply(lambda x: x == 'True')

In [210]:
df_res[df_res.flag_correct ==False][['question', 'provided_answer', 'answer']]

,question,provided_answer,answer
2,"Which film whose director is younger, Charge I...",Charge It To Me.,Danger: Diabolik
5,When is the composer of film Sruthilayalu 's b...,The composer of film Sruthilayalu is K. V. Mah...,14 March 1918
11,Are Alison Skipper and Diane Gilliam Fisher fr...,No.,yes
13,Which country the performer of song Soldier (N...,Canada.,Canadian
17,When did Fatima Bint Mubarak Al Ketbi's husban...,The text does not mention the date of Fatima b...,2 November 2004
19,"Are both businesses, Vakıfbank and Infopro Sdn...","Yes, both businesses are located in Malaysia.",no
23,"Which film was released more recently, Die sch...",Die schöne Lurette (1960) was released earlier...,Sabhash
27,When did the director of film An International...,The information about the director of the film...,10 August 1960
36,Who is the mother-in-law of Andrew Murray (Sco...,Christina Bruce.,"Marjorie, Countess of Carrick"
39,When did the performer of song A Pretty Girl M...,There is no information about the performer of...,"June 22, 1969"


## Results Aggregation

In [ ]:
# Filter df_res if we want Correctly answered samples or not

### LDS

In [213]:
avg_lds = {k:[] for k in [list(m.keys())[0] for m in eval(df_res.lds.loc[0])[0][0]]}
for i in df_res.index: 
    for elem in eval(df_res.loc[i].lds)[0][0]: 
        avg_lds[list(elem.keys())[0]].append(list(elem.values())[0])


In [200]:
list(elem.values())

[0.7072302558398219]

In [ ]:
for i in df_res.index


AttributeError: 'dict' object has no attribute 'key'

In [85]:
# all_values_mean = {'ContextCite128' : [], 'ContextCite256' : [], 'ContextCite512': [],
#                    'FM_Weights128' : [], 'FM_Weights256' : [], 'FM_Weights512': [],
#                    'LOO' : [], 'ARC-JSD' : []}
general = {}
for metric in ['lds', 'r2']:
    all_values_mean = {'ContextCite128' : [], 'ContextCite256' : [], 'ContextCite512': [],
                   'FM_Weights128' : [], 'FM_Weights256' : [], 'FM_Weights512': [],
                   'LOO' : [], 'ARC-JSD' : []}
    for elem in df[metric]:
        inner_list = eval(elem)[0]
        for d in inner_list:
           {all_values_mean[key].append(value) for key, value in d.items()}
    general[metric] = all_values_mean

In [89]:
all_values_mean = {'ContextCite128' : [], 'ContextCite256' : [], 'ContextCite512': [],
                   'FM_Weights128' : [], 'FM_Weights256' : [], 'FM_Weights512': [],
                   'LOO' : [], 'ARC-JSD' : []}
metric = 'r2'
for elem in df[metric]:
    inner_list = eval(elem)[0]
    for d in inner_list:
        {all_values_mean[key].append(value) for key, value in d.items()}

res = {k:np.mean(v) for k,v in all_values_mean.items()}
res
    

{'ContextCite128': np.float64(0.6575588702541534),
 'ContextCite256': np.float64(0.6575588702541534),
 'ContextCite512': np.float64(0.6575588702541534),
 'FM_Weights128': np.float64(-0.462296831279265),
 'FM_Weights256': np.float64(-0.462296831279265),
 'FM_Weights512': np.float64(-0.462296831279265),
 'LOO': np.float64(0.6575588702541534),
 'ARC-JSD': np.float64(0.6575588702541534)}

In [ ]:
general_mean = {}

res = {k:np.mean(v) for k,v in all_values_mean.items()}
res

{'ContextCite128': np.float64(0.7151445872497226),
 'ContextCite256': np.float64(0.725790669177992),
 'ContextCite512': np.float64(0.7303898646421513),
 'FM_Weights128': np.float64(0.11665627039820357),
 'FM_Weights256': np.float64(0.11665627039820357),
 'FM_Weights512': np.float64(0.11665627039820357),
 'LOO': np.float64(0.6555227255469859),
 'ARC-JSD': np.float64(0.6713302550419226)}